In [25]:
# Import library
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

import re
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

from mlxtend.feature_selection import SequentialFeatureSelector as sfs
import sklearn.model_selection as ms
import statsmodels.api as sm 

os.chdir('/Users/macbook/Downloads')
os.getcwd()

df = pd.read_csv("Movado.csv")

The goal is to understand how efficient each marketing channel is to drive orders and sales
- Develop statistical models to evaluate marketing efficiency
- Models should be tested at both daily and weekly levels
- Models should consider seasonality
- Modelling code should be clean with necessary explanations and results should be reproducible
- The deliverable should include modeling codes and a writeup to describe how you understand the data and problem,how do you select variables, how you choose your model, and what the results look like and how we can use the results to direct future marketing decision-making

In [26]:
df.head()

,Date,orders,sales,S/O,Cost_Criteo,Cost_Email,Cost_FB_P,Cost_FB_R,Cost_GDN_P,Cost_GDN_R,Cost_Pint_P,Cost_Pint_R,Cost_Podcasts,Cost_SEM_B,Cost_SEM_NB,Cost_TV
0,1/1/16,45.428571,5324.794286,117.212453,0.0,0,0.0,49.982143,33.241429,176.453571,0.0,0.0,0.0,117.282143,118.801429,0.0
1,1/2/16,48.642857,5790.447857,119.040044,0.0,0,0.0,39.867857,18.130714,71.442143,0.0,0.0,0.0,132.391429,71.215000,0.0
2,1/3/16,47.785714,5541.456429,115.964709,0.0,0,0.0,42.558571,21.433571,84.910714,0.0,0.0,0.0,147.811429,53.144286,0.0
3,1/4/16,81.500000,9972.320714,122.359763,0.0,0,0.0,45.019286,18.827143,65.668571,0.0,0.0,0.0,161.435000,90.473571,0.0
4,1/5/16,79.714286,9530.935714,119.563710,0.0,0,0.0,39.481429,15.142857,66.080000,0.0,0.0,0.0,190.327857,110.420714,0.0


In [27]:
df.shape

(1308, 16)

In [28]:
df.dtypes

Date              object
orders           float64
sales            float64
S/O              float64
Cost_Criteo      float64
Cost_Email         int64
Cost_FB_P        float64
Cost_FB_R        float64
Cost_GDN_P       float64
Cost_GDN_R       float64
Cost_Pint_P      float64
Cost_Pint_R      float64
Cost_Podcasts    float64
Cost_SEM_B       float64
Cost_SEM_NB      float64
Cost_TV          float64
dtype: object

In [29]:
# Find numeber of missing cell 
df.isnull().sum()

Date               0
orders             0
sales              0
S/O              577
Cost_Criteo        0
Cost_Email         0
Cost_FB_P          0
Cost_FB_R          0
Cost_GDN_P         0
Cost_GDN_R         0
Cost_Pint_P        0
Cost_Pint_R        0
Cost_Podcasts      0
Cost_SEM_B         0
Cost_SEM_NB        0
Cost_TV            0
dtype: int64

In [30]:
# Convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

In [31]:
df.dtypes

Date             datetime64[ns]
orders                  float64
sales                   float64
S/O                     float64
Cost_Criteo             float64
Cost_Email                int64
Cost_FB_P               float64
Cost_FB_R               float64
Cost_GDN_P              float64
Cost_GDN_R              float64
Cost_Pint_P             float64
Cost_Pint_R             float64
Cost_Podcasts           float64
Cost_SEM_B              float64
Cost_SEM_NB             float64
Cost_TV                 float64
dtype: object

In [32]:
# Few columns have large of amount of zeros, count how many zeros are in each column
(df == 0).astype(int).sum(axis=0)

Date                0
orders              0
sales               0
S/O                 0
Cost_Criteo       176
Cost_Email       1308
Cost_FB_P          34
Cost_FB_R           0
Cost_GDN_P        832
Cost_GDN_R          8
Cost_Pint_P        29
Cost_Pint_R       256
Cost_Podcasts     579
Cost_SEM_B          0
Cost_SEM_NB         2
Cost_TV           673
dtype: int64

In [33]:
# Cost email column has all 0, drop
df['Cost_Email'].value_counts()
df = df.drop(['Cost_Email'], axis=1).drop(['S/O'], axis = 1)

# Calculate purchase price per order, call it  'price'
df['Price'] = df['sales']/df['orders']

In [34]:
# get the week number in a year, Month, and Season based on Date
df['Week_Num'] = df['Date'].dt.week
df['Month'] = df['Date'].dt.month
df['Weekday'] = df['Date'].dt.weekday

# setup a dictionary where 1 = spring, 2 = summer, 3 = fall and 4 = winter
seasons = [4, 4, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4]
month_to_season = dict(zip(range(1,13), seasons))
month_to_season 
df['Season'] = df.Month.map(month_to_season) 

In [35]:
df.sample(5)

,Date,orders,sales,Cost_Criteo,Cost_FB_P,Cost_FB_R,Cost_GDN_P,Cost_GDN_R,Cost_Pint_P,Cost_Pint_R,Cost_Podcasts,Cost_SEM_B,Cost_SEM_NB,Cost_TV,Price,Week_Num,Month,Weekday,Season
1033,2018-10-30,64.642857,8317.371429,26.792508,1302.086429,98.127857,0.000000,2.257143,36.065000,10.179286,258.857143,73.840714,43.911429,426.821429,128.666519,44,10,1,3
466,2017-04-11,53.214286,7236.928571,53.256420,283.237143,112.710714,0.000000,10.542143,48.482857,17.235714,1200.815714,68.654286,27.425000,0.000000,135.995973,15,4,1,1
951,2018-08-09,53.071429,7228.480000,11.045585,852.254286,107.713571,0.000000,2.114286,64.240714,13.205714,0.000000,83.242857,39.433571,610.239286,136.202853,32,8,3,2
98,2016-04-08,40.571429,5078.821429,0.000000,789.436429,135.289286,42.102143,101.292857,39.199286,0.000000,0.000000,49.527857,58.759286,0.000000,125.182218,14,4,4,1
1011,2018-10-08,68.500000,9637.665714,29.517514,883.740000,79.254286,0.000000,1.246429,35.382857,12.230000,2536.765000,88.462143,52.004286,736.707143,140.695850,41,10,0,3


In [36]:
df.describe()

,orders,sales,Cost_Criteo,Cost_FB_P,Cost_FB_R,Cost_GDN_P,Cost_GDN_R,Cost_Pint_P,Cost_Pint_R,Cost_Podcasts,Cost_SEM_B,Cost_SEM_NB,Cost_TV,Price,Week_Num,Month,Weekday,Season
count,1308.000000,1308.000000,1308.000000,1308.000000,1308.000000,1308.000000,1308.000000,1308.000000,1308.000000,1308.000000,1308.000000,1308.000000,1308.000000,1308.000000,1308.000000,1308.000000,1308.00000,1308.000000
mean,95.186435,12198.738598,40.700948,1715.585807,463.208914,7.164540,21.495212,114.542351,20.382497,612.198295,116.838139,91.627252,393.866474,127.625450,24.805046,6.116208,3.00000,2.431957
std,125.261276,16264.956604,57.257554,3373.736007,948.556932,12.597595,29.153344,193.691279,30.774349,1012.008789,109.665705,141.635854,858.147648,12.707870,14.780342,3.387986,2.00153,1.146147
min,22.714286,2608.207143,0.000000,0.000000,13.970714,0.000000,0.000000,0.000000,0.000000,0.000000,36.971429,0.000000,0.000000,89.479635,1.000000,1.000000,0.00000,1.000000
25%,51.839286,6686.075715,11.784760,500.481429,149.430714,0.000000,3.420714,47.506964,7.487500,0.000000,69.173214,41.034464,0.000000,122.331269,12.000000,3.000000,1.00000,1.000000
50%,61.321429,7960.246071,26.127846,852.077857,206.923929,0.000000,10.297857,67.836071,13.459643,119.500000,83.801071,62.875714,0.000000,128.554492,24.000000,6.000000,3.00000,2.000000
75%,83.142857,10090.348748,45.582375,1409.314108,390.141071,12.412321,28.890714,105.314286,20.578214,829.123929,110.189821,89.468393,467.773214,135.922348,37.000000,9.000000,5.00000,4.000000
max,1879.071429,243411.206400,666.212341,39357.307140,17647.035000,91.780714,333.711429,2243.424286,258.049286,6943.932563,1178.240714,3040.415714,8780.560714,171.989987,53.000000,12.000000,6.00000,4.000000


In [37]:
delete_row = df[df["Cost_Pint_P"]>1000].index
df_pint = df.drop(delete_row)

## Machine Learning Models - Target: Sales

### Train Test Split

In [38]:
df = df.drop(['Date'], axis=1).drop (['orders'], axis =1)
X = df.drop('sales', axis=1)
y = df['sales']

X_list = list(X.columns)
print('='*50)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (1046, 16)
Training Labels Shape: (1046,)
Testing Features Shape: (262, 16)
Testing Labels Shape: (262,)


## Linear Regression 

In [39]:
regressor = LinearRegression()  
regressor.fit(X_train, y_train) #training the algorithm

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [40]:
y_train = y_train.ravel()
y_test = y_test.ravel()

print('Training dataset shape:', X_train.shape, y_train.shape)
print('Testing dataset shape:', X_test.shape, y_test.shape)
print('-'*50)
print("R^2 for train set: %f" %regressor.score(X_train, y_train))
print('-'*50)
print("R^2 for test  set: %f" %regressor.score(X_test, y_test))

Training dataset shape: (1046, 16) (1046,)
Testing dataset shape: (262, 16) (262,)
--------------------------------------------------
R^2 for train set: 0.927274
--------------------------------------------------
R^2 for test  set: 0.879028


In [41]:
X_add_const = sm.add_constant(X_train)
ols = sm.OLS(y_train, X_add_const)
ans = ols.fit()
print(ans.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.927
Model:                            OLS   Adj. R-squared:                  0.926
Method:                 Least Squares   F-statistic:                     820.0
Date:                Tue, 03 Mar 2020   Prob (F-statistic):               0.00
Time:                        21:34:15   Log-Likelihood:                -10305.
No. Observations:                1046   AIC:                         2.064e+04
Df Residuals:                    1029   BIC:                         2.073e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -1555.0144   1620.835     -0.959

In [42]:
print(regressor.coef_)

[ 2.77359409e+01  2.07315121e+00  2.68585720e+00 -3.21570081e+01
 -1.46328931e+01 -6.14584109e+00  2.09893304e+01 -8.48618609e-02
  5.00354671e+01  2.21477073e+01 -6.43286376e-01  3.23894424e+01
 -3.22631327e+01  4.56438223e+01 -1.37534458e+02 -8.27898863e+02]


In [43]:
print(np.std(X, 0)*regressor.coef_)

Cost_Criteo      1587.484947
Cost_FB_P        6991.590710
Cost_FB_R        2546.714391
Cost_GDN_P       -404.946088
Cost_GDN_R       -426.434665
Cost_Pint_P     -1189.940690
Cost_Pint_R       645.686008
Cost_Podcasts     -85.848114
Cost_SEM_B       5485.076822
Cost_SEM_NB      3135.710085
Cost_TV          -551.823628
Price             411.443447
Week_Num         -476.677827
Month             154.581524
Weekday          -275.174044
Season           -948.530944
dtype: float64


In [44]:
std = ols.exog.std(0)
std[0] = 1
tt = ans.t_test(np.diag(std))
print(tt.summary())


                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0         -1555.0144   1620.835     -0.959      0.338   -4735.533    1625.504
c1          1573.8530    231.744      6.791      0.000    1119.109    2028.597
c2          7306.2464    272.751     26.787      0.000    6771.035    7841.458
c3          2464.2295    307.485      8.014      0.000    1860.860    3067.599
c4          -409.1797    196.803     -2.079      0.038    -795.361     -22.998
c5          -419.4011    236.967     -1.770      0.077    -884.396      45.594
c6         -1170.2495    217.787     -5.373      0.000   -1597.607    -742.892
c7           625.6885    272.868      2.293      0.022      90.248    1161.129
c8           -85.3695    161.942     -0.527      0.598    -403.144     232.405
c9          5655.9487    353.658     15.993      0.0

### Forward Feature Selection with Linear Regression 

In [45]:
feature_selection = range(1, 11)

features = []
train_scores = []
test_scores = []

for k_features in feature_selection:
    ols = LinearRegression()
    sfs1 = sfs(ols,
               k_features=k_features,
               forward=True,
               floating=False,
               verbose=0,
               scoring='r2',
               cv=5,
               n_jobs=-1)
    sfs1 = sfs1.fit(np.array(X), np.array(df['sales']))
    feat_cols = list(sfs1.k_feature_idx_)
    
    print(k_features)
    features.append(feat_cols)
    print('Features: ' + str(feat_cols))
    ols.fit(X.iloc[:,feat_cols], df['sales'])
    train_scores.append(ols.score(X.iloc[:,feat_cols], df['sales']))
    print('Train Score: ' + str(train_scores[-1]))
    ms_k3 = ms.KFold(n_splits=3)
    test_scores.append(np.mean(ms.cross_val_score(estimator=ols, X=X.iloc[:, feat_cols], y=y, cv=ms_k3)))
    print('Test Score: ' + str(test_scores[-1]))

1
Features: [8]
Train Score: 0.7771835938556669
Test Score: 0.7580701132964743
2
Features: [1, 8]
Train Score: 0.8907827116632381
Test Score: 0.8805559480634052
3
Features: [1, 8, 9]
Train Score: 0.9060809917269949
Test Score: 0.870697955150867
4
Features: [1, 6, 8, 9]
Train Score: 0.9120478053553042
Test Score: 0.8814776029372923
5
Features: [1, 6, 8, 9, 15]
Train Score: 0.9143380766149561
Test Score: 0.882623270915345
6
Features: [0, 1, 6, 8, 9, 15]
Train Score: 0.9185036926093966
Test Score: 0.874784374595805
7
Features: [0, 1, 6, 8, 9, 14, 15]
Train Score: 0.9189910845465401
Test Score: 0.8763095734302392
8
Features: [0, 1, 6, 8, 9, 12, 14, 15]
Train Score: 0.9190453797716807
Test Score: 0.8764860076578019
9
Features: [0, 1, 6, 8, 9, 12, 13, 14, 15]
Train Score: 0.9190556915963618
Test Score: 0.8762377174612953
10
Features: [0, 1, 6, 7, 8, 9, 12, 13, 14, 15]
Train Score: 0.9190701724691367
Test Score: 0.8756632219658019


In [46]:
X.iloc[:,features[-1]].columns

Index(['Cost_Criteo', 'Cost_FB_P', 'Cost_Pint_R', 'Cost_Podcasts',
       'Cost_SEM_B', 'Cost_SEM_NB', 'Week_Num', 'Month', 'Weekday', 'Season'],
      dtype='object')

The 6 most important original features to sales are
- Cost_Criteo
- Cost_FB_P
- Cost_Pint_R
- Cost_Podcasts
- Cost_SEM_B
- Cost_SEM_NB

In [47]:
X1 = df[['Cost_Criteo','Cost_FB_P','Cost_Pint_R', 'Cost_Podcasts', 'Cost_SEM_B', 'Cost_SEM_NB']]
y1 = df['sales']

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=0)
print('Training Features Shape:', X1_train.shape)
print('Training Labels Shape:', y1_train.shape)
print('Testing Features Shape:', X1_test.shape)
print('Testing Labels Shape:', y1_test.shape)

Training Features Shape: (1046, 6)
Training Labels Shape: (1046,)
Testing Features Shape: (262, 6)
Testing Labels Shape: (262,)


In [48]:
regressor = LinearRegression()  
regressor.fit(X1_train, y1_train) #training the algorithm

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [49]:
y1_train = y1_train.ravel()
y1_test = y1_test.ravel()

print('Training dataset shape:', X1_train.shape, y1_train.shape)
print('Testing dataset shape:', X1_test.shape, y1_test.shape)
print('-'*50)
print("R^2 for train set: %f" %regressor.score(X1_train, y1_train))
print('-'*50)
print("R^2 for test  set: %f" %regressor.score(X1_test, y1_test))

Training dataset shape: (1046, 6) (1046,)
Testing dataset shape: (262, 6) (262,)
--------------------------------------------------
R^2 for train set: 0.916771
--------------------------------------------------
R^2 for test  set: 0.904847


In [50]:
X1_add_const = sm.add_constant(X1_train)
ols = sm.OLS(y1_train, X1_add_const)
ans = ols.fit()
print(ans.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.917
Model:                            OLS   Adj. R-squared:                  0.916
Method:                 Least Squares   F-statistic:                     1907.
Date:                Tue, 03 Mar 2020   Prob (F-statistic):               0.00
Time:                        21:37:00   Log-Likelihood:                -10375.
No. Observations:                1046   AIC:                         2.076e+04
Df Residuals:                    1039   BIC:                         2.080e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          -471.5978    240.606     -1.960